In [30]:
!pip install -r ./Poc/requirements.txt

     |████████████████████████████████| 43 kB 1.5 MB/s eta 0:00:011


  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16182 sha256=b874093b3328b15791ec0216ba8b88738842ebe5a36ccbfeffe0fbe5c2f98240
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from transformers import BertTokenizerFast, BertForPreTraining,BertForTokenClassification ,Trainer, TrainingArguments
import os
import torch 
import datasets

In [2]:
torch.cuda.is_available()

True

In [5]:
from array import array
import numpy as np
def loadData():
    list_subfolders_with_paths = [f.path for f in os.scandir("./annotation3/") if f.is_dir()]
    data = {"texts": [],"labels":[]}
    for i in list_subfolders_with_paths:
    #print("new File")
        texts=[]
        labels=[]
        with open(f"{i}/admin.conll","r", encoding="utf-8-sig") as file: 
            for j in  file.readlines():
                if(j.strip() == ""):
                    continue
                sp = j.split(" ")
                if("null" in sp[1].rstrip("\n")):
                    print(i)
                texts.append(sp[0])
                labels.append(sp[1].rstrip("\n"))
                #print(f"Text: {sp[0]}")
                #print(f"Label: {sp[1]}")
            data["texts"].append((texts))
            data["labels"].append(( labels))
    
    
    return data

In [4]:
from array import array
import numpy as np
def loadData():
    list_subfolders_with_paths = [f.path for f in os.scandir("./annotation3/") if f.is_dir()]
    data = {"texts": [],"labels":[]}
    for i in list_subfolders_with_paths:
    #print("new File")
        texts=[]
        labels=[]
        with open(f"{i}/admin.conll","r", encoding="utf-8-sig") as file: 
            for j in  file.readlines():
                if(j.strip() == ""):
                    if( "SENTENCE_END" in labels[-1] ):
                        data["texts"].append((texts))
                        data["labels"].append(( labels))
                        texts=[]
                        labels=[]
                    continue
                sp = j.split(" ")
                if("null" in sp[1].rstrip("\n")):
                    print(i)
                texts.append(sp[0])
                labels.append(sp[1].rstrip("\n"))
                #print(f"Text: {sp[0]}")
                #print(f"Label: {sp[1]}")
            data["texts"].append((texts))
            data["labels"].append(( labels))
    
    
    return data

In [6]:
from datasets import Dataset , ClassLabel ,Features ,Sequence ,Value

In [7]:
traindata = loadData()
texts = traindata["texts"]
tags = traindata["labels"]

In [8]:
from collections import Counter
x = [i[2:] if(len(i) > 1) else i for i in list(sum(tags, []))]
di = dict(Counter(x))

z = {}

for i in di.keys():
    if i == "O":
        continue
    z[i] = round ((di[i]/(len(x)-di["O"]))*100,2)
    

z

{'PERSON_FIRSTNAME': 13.1,
 'PERSON_LASTNAME': 14.08,
 'PERSON_AGE': 6.64,
 'SENTENCE_END': 43.22,
 'LOCATION_CITY': 7.2,
 'ORGANIZATION': 6.64,
 'LOCATION_COUNTRY': 0.94,
 'LOCATION_STREET': 1.5,
 'LOCATION_ASSOSIATION': 1.96,
 'LOCATION': 3.46,
 'TELE_NUMMER': 1.26}

In [11]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [12]:
list(unique_tags)

['B-TELE_NUMMER',
 'I-ORGANIZATION',
 'B-LOCATION_COUNTRY',
 'B-ORGANIZATION',
 'I-LOCATION_ASSOSIATION',
 'I-PERSON_FIRSTNAME',
 'B-LOCATION_STREET',
 'B-PERSON_AGE',
 'I-TELE_NUMMER',
 'O',
 'I-SENTENCE_END',
 'I-LOCATION_STREET',
 'I-LOCATION',
 'B-LOCATION_ASSOSIATION',
 'B-LOCATION',
 'B-PERSON_FIRSTNAME',
 'B-PERSON_LASTNAME',
 'B-LOCATION_CITY',
 'B-SENTENCE_END',
 'I-PERSON_AGE',
 'I-LOCATION_CITY']

In [13]:
dataset = Dataset.from_dict(loadData(),features=Features( {"texts": Sequence(Value("string")),'labels':Sequence(ClassLabel(names=list(unique_tags),num_classes =len(unique_tags)))}))

In [15]:
dataset = dataset.train_test_split(test_size=0.2)

In [16]:
{str(v): k for k, v in (dataset["train"].features["labels"].feature)._str2int.items()}

{'0': 'B-TELE_NUMMER',
 '1': 'I-ORGANIZATION',
 '2': 'B-LOCATION_COUNTRY',
 '3': 'B-ORGANIZATION',
 '4': 'I-LOCATION_ASSOSIATION',
 '5': 'I-PERSON_FIRSTNAME',
 '6': 'B-LOCATION_STREET',
 '7': 'B-PERSON_AGE',
 '8': 'I-TELE_NUMMER',
 '9': 'O',
 '10': 'I-SENTENCE_END',
 '11': 'I-LOCATION_STREET',
 '12': 'I-LOCATION',
 '13': 'B-LOCATION_ASSOSIATION',
 '14': 'B-LOCATION',
 '15': 'B-PERSON_FIRSTNAME',
 '16': 'B-PERSON_LASTNAME',
 '17': 'B-LOCATION_CITY',
 '18': 'B-SENTENCE_END',
 '19': 'I-PERSON_AGE',
 '20': 'I-LOCATION_CITY'}

In [17]:
dataset["train"][0]

{'texts': ['SAMUEL',
  'YEBOAH',
  'STARB',
  '1991',
  'BEI',
  'BRANDANSCHLAG',
  'Plakate',
  'erinnern',
  'an',
  'ermordeten',
  'Flüchtling',
  'Ursula',
  'Quack',
  'vom',
  'Flüchtlingsrat',
  'und',
  'Roland',
  'Röder',
  'von',
  'der',
  'Aktion3',
  '.',
  'Welt',
  'Saar',
  'beim',
  'Anbringen',
  'des',
  'ersten',
  'Plakats',
  'Foto',
  ':',
  'Dirk',
  'Guldner',
  '-',
  'guldner.de',
  '13.08.2021',
  '-',
  '19',
  ':',
  '38',
  'Uhr',
  'Saarlouis',
  '–',
  'Gigantische',
  'Plakate',
  'gegen',
  'das',
  'Vergessen',
  '!',
  'In',
  'der',
  'Nacht',
  'zum',
  '19',
  '.',
  'September',
  '1991',
  'verbrannte',
  'Samuel',
  'Yeboah',
  '(',
  '†',
  '27',
  ')',
  'bei',
  'einem',
  'Anschlag',
  'in',
  'Fraulautern',
  '.',
  'Seit',
  'letztem',
  'Jahr',
  'ermittelt',
  'die',
  'Bundesanwaltschaft',
  'wegen',
  'Mordes',
  ',',
  'geht',
  'von',
  'einem',
  'rechtsextremen',
  'Motiv',
  'aus',
  '(',
  'BILD',
  'berichtete',
  ')',
  '.'

In [18]:
label_names = dataset["train"].features[f"labels"].feature.names

In [19]:
len(label_names)

21

In [20]:
#Load Model and Tokenizer
from transformers import AutoTokenizer

tokenizer = BertTokenizerFast.from_pretrained('deepset/gbert-large')
model = BertForTokenClassification.from_pretrained('deepset/gbert-large', num_labels=len(label_names) )

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/gbert-large were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from t

In [21]:
model.config.label2id =  (dataset["train"].features["labels"].feature)._str2int 
model.config.id2label =  {str(v): k for k, v in (dataset["train"].features["labels"].feature)._str2int.items()}

In [22]:
def tokenize_and_align_labels(examples):
    #print(examples.keys())
    tokenized_inputs = tokenizer(examples["texts"],max_length= 1024, truncation=True, is_split_into_words=True) # set max length 1024

    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [23]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['texts', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39
    })
    test: Dataset({
        features: ['texts', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [25]:
def split(examples):
    dic ={"input_ids":[],"token_type_ids":[],"attention_mask":[],"labels":[]}
    for j in examples:
        if j == "texts":
            continue
        for i in range(len(examples["input_ids"])):
            arr = np.array_split(examples[j][i],2)
            dic[j].append(arr[0])
            dic[j].append(arr[1])
    return dic

In [26]:
test = tokenized_datasets.map(split,remove_columns=["input_ids","token_type_ids","attention_mask","labels","texts"], batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
full_train_dataset = test["train"]
full_eval_dataset = test["test"]

In [28]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [31]:
from datasets import load_metric
import numpy as np
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [32]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [34]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=100,
    weight_decay=0.01,
)

In [35]:
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset,compute_metrics=compute_metrics, data_collator=data_collator,
    tokenizer=tokenizer)

In [36]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 78
  Num Epochs = 100
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 1300


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Location F1,Location Assosiation F1,Location City F1,Location Country F1,Location Street F1,Organization F1,Person Age F1,Person Firstname F1,Person Lastname F1,Sentence End F1
1,No log,0.522151,0.000000,0.000000,0.000000,0.884777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,No log,0.263374,0.970000,0.527174,0.683099,0.941637,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.674419,0.953757
3,No log,0.140578,0.895833,0.817935,0.855114,0.968412,0.000000,0.000000,0.557377,0.000000,0.000000,0.000000,0.945455,0.918367,0.838095,0.965714
4,No log,0.106336,0.823678,0.888587,0.854902,0.970217,0.000000,0.000000,0.676471,0.000000,0.000000,0.285714,0.928571,0.953271,0.890756,0.963173
5,No log,0.086727,0.833753,0.899457,0.865359,0.973526,0.000000,0.181818,0.779661,0.000000,0.000000,0.300000,1.000000,0.990291,0.905983,0.968839
6,No log,0.079236,0.837838,0.926630,0.880000,0.976835,0.000000,0.666667,0.821429,1.000000,0.222222,0.325581,1.000000,0.971963,0.913793,0.974648
7,No log,0.086754,0.845387,0.921196,0.881664,0.974428,0.000000,0.428571,0.807018,1.000000,0.133333,0.437500,1.000000,0.971963,0.929825,0.971910
8,No log,0.085261,0.866324,0.915761,0.890357,0.975030,0.000000,0.588235,0.814815,0.666667,0.400000,0.487805,1.000000,0.980392,0.939130,0.965517
9,No log,0.077089,0.870324,0.948370,0.907672,0.978941,0.275862,0.736842,0.821429,1.000000,0.285714,0.588235,1.000000,1.000000,0.947368,0.971591
10,No log,0.082003,0.856790,0.942935,0.897801,0.978039,0.181818,0.736842,0.821429,0.666667,0.500000,0.555556,1.000000,0.990291,0.938053,0.971751


***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Running Evaluation *****
  Num examples = 20
  Batch size = 6
***** Ru

TrainOutput(global_step=1300, training_loss=0.02402842131944803, metrics={'train_runtime': 433.2059, 'train_samples_per_second': 18.005, 'train_steps_per_second': 3.001, 'total_flos': 5100007352740956.0, 'train_loss': 0.02402842131944803, 'epoch': 100.0})

In [37]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 20
  Batch size = 6


{'eval_loss': 0.1489889919757843,
 'eval_overall_precision': 0.8787878787878788,
 'eval_overall_recall': 0.9456521739130435,
 'eval_overall_f1': 0.9109947643979057,
 'eval_overall_accuracy': 0.9786401925391095,
 'eval_LOCATION_f1': 0.26666666666666666,
 'eval_LOCATION_ASSOSIATION_f1': 0.7058823529411765,
 'eval_LOCATION_CITY_f1': 0.8301886792452831,
 'eval_LOCATION_COUNTRY_f1': 0.6666666666666666,
 'eval_LOCATION_STREET_f1': 0.4,
 'eval_ORGANIZATION_f1': 0.625,
 'eval_PERSON_AGE_f1': 1.0,
 'eval_PERSON_FIRSTNAME_f1': 0.9902912621359222,
 'eval_PERSON_LASTNAME_f1': 0.9473684210526315,
 'eval_SENTENCE_END_f1': 0.9746478873239437,
 'eval_runtime': 0.2839,
 'eval_samples_per_second': 70.44,
 'eval_steps_per_second': 14.088,
 'epoch': 100.0}

In [38]:
model.save_pretrained('./NEWS_NER_2/')

Configuration saved in ./NEWS_NER_2/config.json
Model weights saved in ./NEWS_NER_2/pytorch_model.bin


In [39]:
tokenizer.save_pretrained('./NEWS_TOK_2/')

tokenizer config file saved in ./NEWS_TOK_2/tokenizer_config.json
Special tokens file saved in ./NEWS_TOK_2/special_tokens_map.json


('./NEWS_TOK_2/tokenizer_config.json',
 './NEWS_TOK_2/special_tokens_map.json',
 './NEWS_TOK_2/vocab.txt',
 './NEWS_TOK_2/added_tokens.json',
 './NEWS_TOK_2/tokenizer.json')

In [59]:
{'eval_loss': 0.11371712386608124,
 'eval_overall_precision': 0.8902077151335311,
 'eval_overall_recall': 0.9433962264150944,
 'eval_overall_f1': 0.916030534351145,
 'eval_overall_accuracy': 0.982909090909091,
 'eval_LOCATION_f1': 0.2727272727272727,
 'eval_LOCATION_ASSOSIATION_f1': 0.6666666666666666,
 'eval_LOCATION_CITY_f1': 0.8823529411764706,
 'eval_LOCATION_COUNTRY_f1': 0.5,
 'eval_LOCATION_STREET_f1': 1.0,
 'eval_ORGANIZATION_f1': 0.6086956521739131,
 'eval_PERSON_AGE_f1': 1.0,
 'eval_PERSON_FIRSTNAME_f1': 1.0,
 'eval_PERSON_LASTNAME_f1': 0.9600000000000001,
 'eval_SENTENCE_END_f1': 0.9822064056939502,
 'eval_runtime': 0.2295,
 'eval_samples_per_second': 43.569,
 'eval_steps_per_second': 8.714,
 'epoch': 100.0}

{'eval_loss': 0.11371712386608124,
 'eval_overall_precision': 0.8902077151335311,
 'eval_overall_recall': 0.9433962264150944,
 'eval_overall_f1': 0.916030534351145,
 'eval_overall_accuracy': 0.982909090909091,
 'eval_LOCATION_f1': 0.2727272727272727,
 'eval_LOCATION_ASSOSIATION_f1': 0.6666666666666666,
 'eval_LOCATION_CITY_f1': 0.8823529411764706,
 'eval_LOCATION_COUNTRY_f1': 0.5,
 'eval_LOCATION_STREET_f1': 1.0,
 'eval_ORGANIZATION_f1': 0.6086956521739131,
 'eval_PERSON_AGE_f1': 1.0,
 'eval_PERSON_FIRSTNAME_f1': 1.0,
 'eval_PERSON_LASTNAME_f1': 0.9600000000000001,
 'eval_SENTENCE_END_f1': 0.9822064056939502,
 'eval_runtime': 0.2295,
 'eval_samples_per_second': 43.569,
 'eval_steps_per_second': 8.714,
 'epoch': 100.0}

In [91]:
{'eval_loss': 0.13603897392749786,
 'eval_overall_precision': 0.8922651933701657,
 'eval_overall_recall': 0.930835734870317,
 'eval_overall_f1': 0.9111424541607898,
 'eval_overall_accuracy': 0.9815531091936923,
 'eval_LOCATION_f1': 0.32,
 'eval_LOCATION_ASSOSIATION_f1': 0.47058823529411764,
 'eval_LOCATION_CITY_f1': 0.851851851851852,
 'eval_LOCATION_COUNTRY_f1': 0.8,
 'eval_LOCATION_STREET_f1': 0.7692307692307693,
 'eval_ORGANIZATION_f1': 0.75,
 'eval_PERSON_AGE_f1': 0.9230769230769231,
 'eval_PERSON_FIRSTNAME_f1': 0.9538461538461539,
 'eval_PERSON_LASTNAME_f1': 0.9863013698630138,
 'eval_SENTENCE_END_f1': 0.9915966386554621,
 'eval_TELE_NUMMER_f1': 0.75,
 'eval_runtime': 0.3311,
 'eval_samples_per_second': 60.409,
 'eval_steps_per_second': 12.082,
 'epoch': 100.0}

{'eval_loss': 0.13603897392749786,
 'eval_overall_precision': 0.8922651933701657,
 'eval_overall_recall': 0.930835734870317,
 'eval_overall_f1': 0.9111424541607898,
 'eval_overall_accuracy': 0.9815531091936923,
 'eval_LOCATION_f1': 0.32,
 'eval_LOCATION_ASSOSIATION_f1': 0.47058823529411764,
 'eval_LOCATION_CITY_f1': 0.851851851851852,
 'eval_LOCATION_COUNTRY_f1': 0.8,
 'eval_LOCATION_STREET_f1': 0.7692307692307693,
 'eval_ORGANIZATION_f1': 0.75,
 'eval_PERSON_AGE_f1': 0.9230769230769231,
 'eval_PERSON_FIRSTNAME_f1': 0.9538461538461539,
 'eval_PERSON_LASTNAME_f1': 0.9863013698630138,
 'eval_SENTENCE_END_f1': 0.9915966386554621,
 'eval_TELE_NUMMER_f1': 0.75,
 'eval_runtime': 0.3311,
 'eval_samples_per_second': 60.409,
 'eval_steps_per_second': 12.082,
 'epoch': 100.0}

In [90]:
{'eval_loss': 0.12491893023252487,
 'eval_overall_precision': 0.8971291866028708,
 'eval_overall_recall': 0.9375,
 'eval_overall_f1': 0.9168704156479217,
 'eval_overall_accuracy': 0.9827833572453372,
 'eval_LOCATION_f1': 0.125,
 'eval_LOCATION_ASSOSIATION_f1': 0.5454545454545454,
 'eval_LOCATION_CITY_f1': 0.8333333333333334,
 'eval_LOCATION_COUNTRY_f1': 0.28571428571428575,
 'eval_LOCATION_STREET_f1': 0.7777777777777778,
 'eval_ORGANIZATION_f1': 0.6153846153846153,
 'eval_PERSON_AGE_f1': 0.9818181818181818,
 'eval_PERSON_FIRSTNAME_f1': 0.9642857142857143,
 'eval_PERSON_LASTNAME_f1': 0.9523809523809523,
 'eval_SENTENCE_END_f1': 0.9892473118279571,
 'eval_TELE_NUMMER_f1': 1.0,
 'eval_runtime': 0.9958,
 'eval_samples_per_second': 172.731,
 'eval_steps_per_second': 29.123,
 'epoch': 20.0}

{'eval_loss': 0.12491893023252487,
 'eval_overall_precision': 0.8971291866028708,
 'eval_overall_recall': 0.9375,
 'eval_overall_f1': 0.9168704156479217,
 'eval_overall_accuracy': 0.9827833572453372,
 'eval_LOCATION_f1': 0.125,
 'eval_LOCATION_ASSOSIATION_f1': 0.5454545454545454,
 'eval_LOCATION_CITY_f1': 0.8333333333333334,
 'eval_LOCATION_COUNTRY_f1': 0.28571428571428575,
 'eval_LOCATION_STREET_f1': 0.7777777777777778,
 'eval_ORGANIZATION_f1': 0.6153846153846153,
 'eval_PERSON_AGE_f1': 0.9818181818181818,
 'eval_PERSON_FIRSTNAME_f1': 0.9642857142857143,
 'eval_PERSON_LASTNAME_f1': 0.9523809523809523,
 'eval_SENTENCE_END_f1': 0.9892473118279571,
 'eval_TELE_NUMMER_f1': 1.0,
 'eval_runtime': 0.9958,
 'eval_samples_per_second': 172.731,
 'eval_steps_per_second': 29.123,
 'epoch': 20.0}